# Utilizar modelo como Model as a Service

Para ejecutar correctamente esta libreta debemos haber ejecudato el script **server.py** que permite levantar el servidor Flask con el modelo. 

In [2]:
# Importar librerías
import requests
import pandas as pd

## Primero probamos con datos reales

In [5]:
# Convertir los datos a un DataFrame
df_input = pd.read_parquet("../../../data/processed/X_test.parquet")[:10]
df_output = pd.read_parquet("../../../data/processed/y_test.parquet")[:10]

# Convertimos los datos de entrada en diccionario
dict_input = df_input.to_dict()

# Inspeccionamos los datos
dict_input

{'ORIGEN': {0: 1, 1: 2, 2: 2, 3: 2, 4: 1, 5: 1, 6: 2, 7: 2, 8: 2, 9: 2},
 'SECTOR': {0: 4, 1: 4, 2: 4, 3: 9, 4: 4, 5: 4, 6: 4, 7: 4, 8: 4, 9: 12},
 'ENTIDAD_UM': {0: 14,
  1: 29,
  2: 1,
  3: 25,
  4: 22,
  5: 17,
  6: 9,
  7: 14,
  8: 24,
  9: 3},
 'SEXO': {0: 1, 1: 2, 2: 2, 3: 1, 4: 1, 5: 2, 6: 2, 7: 2, 8: 1, 9: 2},
 'ENTIDAD_NAC': {0: 14,
  1: 29,
  2: 1,
  3: 25,
  4: 22,
  5: 17,
  6: 9,
  7: 14,
  8: 24,
  9: 3},
 'ENTIDAD_RES': {0: 14,
  1: 29,
  2: 1,
  3: 25,
  4: 22,
  5: 17,
  6: 9,
  7: 14,
  8: 24,
  9: 3},
 'MUNICIPIO_RES': {0: 39,
  1: 41,
  2: 1,
  3: 1,
  4: 14,
  5: 7,
  6: 4,
  7: 93,
  8: 28,
  9: 3},
 'TIPO_PACIENTE': {0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1},
 'INTUBADO': {0: -2,
  1: -2,
  2: -2,
  3: -2,
  4: -2,
  5: -2,
  6: -2,
  7: -2,
  8: -2,
  9: -2},
 'NEUMONIA': {0: 2, 1: 2, 2: 2, 3: 2, 4: 2, 5: 2, 6: 2, 7: 2, 8: 2, 9: 2},
 'EDAD': {0: 51, 1: 25, 2: 31, 3: 40, 4: 12, 5: 1, 6: 35, 7: 22, 8: 11, 9: 45},
 'NACIONALIDAD': {0: 1, 1: 1, 2: 

In [18]:
# Enviar la solicitud POST al servidor Flask
response = requests.post('http://localhost:5000/predict', json=dict_input)

# Código para imprimir predicciones del modelo o identificar errores en caso de que los haya
if response.text:
    try:
        predictions = response.json()
        print(f"Predicciones del modelo: {predictions}")
        print(f"Valores reales: {[x[0] for x in df_output.values]}")
    except Exception as e:
        print(f"Error al decodificar JSON: {e}")
else:
    print("Respuesta vacía del servidor")

Predicciones del modelo: [2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
Valores reales: [2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


## Ahora probamos con los datos de mi amigo hipotético

Le platiqué a un amigo sobre el proyecto que estabamos desarrollando para la materia de aprendizaje automático y su hipocondría le hizo pedirme utilizar el modelo para diagnosticarlo. Le expliqué que mas bien el modelo predice si una persona vivirá o morirá dependiendo de sus características demográficas, comorbilidades y algunos otros datos. No le importó y procedió a proveerme sus datos. El resultado fue el siguiente:

In [6]:
# Cargar los archivos Excel
catalogos_path = "../../../references/processed/Catalogos.xlsx"
catalogos = pd.ExcelFile(catalogos_path)

# Mostrar las hojas de los archivos para entender su estructura
catalogos.sheet_names

['Catálogo ORIGEN',
 'Catálogo SECTOR',
 'Catálogo SEXO',
 'Catálogo TIPO_PACIENTE',
 'Catálogo SI_NO',
 'Catálogo NACIONALIDAD',
 'Catálogo RESULTADO_LAB',
 'Catálogo RESULTADO_ANTIGENO',
 'Catálogo CLASIFICACION_FINAL',
 'Catálogo de ENTIDADES',
 'Catálogo MUNICIPIOS',
 'Catálogo PAISES']

In [7]:
# Convertir catálogo en diccionario de listas
catalogs = {
    'ORIGEN': catalogos.parse('Catálogo ORIGEN')['CLAVE'].tolist(),
    'SECTOR': catalogos.parse('Catálogo SECTOR')['CLAVE'].tolist(),
    'ENTIDAD_UM': catalogos.parse('Catálogo de ENTIDADES')['CLAVE_ENTIDAD'].tolist(),
    'SEXO': catalogos.parse('Catálogo SEXO')['CLAVE'].tolist(),
    'ENTIDAD_NAC': catalogos.parse('Catálogo de ENTIDADES')['CLAVE_ENTIDAD'].tolist(),
    'ENTIDAD_RES': catalogos.parse('Catálogo de ENTIDADES')['CLAVE_ENTIDAD'].tolist(),
    'MUNICIPIO_RES': catalogos.parse('Catálogo MUNICIPIOS')['CLAVE_MUNICIPIO'].tolist(),
    'TIPO_PACIENTE': catalogos.parse('Catálogo TIPO_PACIENTE')['CLAVE'].tolist(),
    'INTUBADO': catalogos.parse('Catálogo SI_NO')['CLAVE'].tolist(),
    'NEUMONIA': catalogos.parse('Catálogo SI_NO')['CLAVE'].tolist(),
    'NACIONALIDAD': catalogos.parse('Catálogo NACIONALIDAD')['CLAVE'].tolist(),
    'EMBARAZO': catalogos.parse('Catálogo SI_NO')['CLAVE'].tolist(),
    'HABLA_LENGUA_INDIG': catalogos.parse('Catálogo SI_NO')['CLAVE'].tolist(),
    'INDIGENA': catalogos.parse('Catálogo SI_NO')['CLAVE'].tolist(),
    'DIABETES': catalogos.parse('Catálogo SI_NO')['CLAVE'].tolist(),
    'EPOC': catalogos.parse('Catálogo SI_NO')['CLAVE'].tolist(),
    'ASMA': catalogos.parse('Catálogo SI_NO')['CLAVE'].tolist(),
    'INMUSUPR': catalogos.parse('Catálogo SI_NO')['CLAVE'].tolist(),
    'HIPERTENSION': catalogos.parse('Catálogo SI_NO')['CLAVE'].tolist(),
    'OTRA_COM': catalogos.parse('Catálogo SI_NO')['CLAVE'].tolist(),
    'CARDIOVASCULAR': catalogos.parse('Catálogo SI_NO')['CLAVE'].tolist(),
    'OBESIDAD': catalogos.parse('Catálogo SI_NO')['CLAVE'].tolist(),
    'RENAL_CRONICA': catalogos.parse('Catálogo SI_NO')['CLAVE'].tolist(),
    'TABAQUISMO': catalogos.parse('Catálogo SI_NO')['CLAVE'].tolist(),
    'OTRO_CASO': catalogos.parse('Catálogo SI_NO')['CLAVE'].tolist(),
    'TOMA_MUESTRA_LAB': catalogos.parse('Catálogo SI_NO')['CLAVE'].tolist(),
    'RESULTADO_LAB': catalogos.parse('Catálogo RESULTADO_LAB')['CLAVE'].tolist(),
    'TOMA_MUESTRA_ANTIGENO': catalogos.parse('Catálogo SI_NO')['CLAVE'].tolist(),
    'RESULTADO_ANTIGENO': catalogos.parse('Catálogo RESULTADO_ANTIGENO')['CLAVE'].tolist(),
    'CLASIFICACION_FINAL': catalogos.parse('Catálogo CLASIFICACION_FINAL')['CLAVE'].tolist(),
    'MIGRANTE': catalogos.parse('Catálogo SI_NO')['CLAVE'].tolist(),
    'PAIS_NACIONALIDAD': catalogos.parse('Catálogo PAISES')['CLAVE_PAIS'].tolist(),
    'PAIS_ORIGEN': catalogos.parse('Catálogo PAISES')['CLAVE_PAIS'].tolist(),
    'UCI': catalogos.parse('Catálogo SI_NO')['CLAVE'].tolist(),
}

In [8]:
import random
from datetime import datetime, timedelta

# Función para generar fecha aleatoria entre un rango de años dado
def random_date(year_start, year_end):
    start_date = datetime(year_start, 1, 1)
    end_date = datetime(year_end, 12, 31)
    random_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))
    return random_date

In [9]:
# Generar fechas
update_date = datetime.today()
symptoms_date = random_date(2020, 2024)
ingreso_date = symptoms_date + timedelta(days=random.randint(1, 5))

In [15]:
# Generar el diccionario con los datos de mi amigo
data_amigo = {
    'ORIGEN': {0: random.choice(catalogs['ORIGEN'])},
    'SECTOR': {0: random.choice(catalogs['SECTOR'])},
    'ENTIDAD_UM': {0: random.choice(catalogs['ENTIDAD_UM'])},
    'SEXO': {0: random.choice(catalogs['SEXO'])},
    'ENTIDAD_NAC': {0: random.choice(catalogs['ENTIDAD_NAC'])},
    'ENTIDAD_RES': {0: random.choice(catalogs['ENTIDAD_RES'])},
    'MUNICIPIO_RES': {0: random.choice(catalogs['MUNICIPIO_RES'])},
    'TIPO_PACIENTE': {0: random.choice(catalogs['TIPO_PACIENTE'])},
    'INTUBADO': {0: random.choice(catalogs['INTUBADO'])},
    'NEUMONIA': {0: random.choice(catalogs['NEUMONIA'])},
    'EDAD': {0: random.randint(0, 120)},
    'NACIONALIDAD': {0: random.choice(catalogs['NACIONALIDAD'])},
    'EMBARAZO': {0: random.choice(catalogs['EMBARAZO'])},
    'HABLA_LENGUA_INDIG': {0: random.choice(catalogs['HABLA_LENGUA_INDIG'])},
    'INDIGENA': {0: random.choice(catalogs['INDIGENA'])},
    'DIABETES': {0: random.choice(catalogs['DIABETES'])},
    'EPOC': {0: random.choice(catalogs['EPOC'])},
    'ASMA': {0: random.choice(catalogs['ASMA'])},
    'INMUSUPR': {0: random.choice(catalogs['INMUSUPR'])},
    'HIPERTENSION': {0: random.choice(catalogs['HIPERTENSION'])},
    'OTRA_COM': {0: random.choice(catalogs['OTRA_COM'])},
    'CARDIOVASCULAR': {0: random.choice(catalogs['CARDIOVASCULAR'])},
    'OBESIDAD': {0: random.choice(catalogs['OBESIDAD'])},
    'RENAL_CRONICA': {0: random.choice(catalogs['RENAL_CRONICA'])},
    'TABAQUISMO': {0: random.choice(catalogs['TABAQUISMO'])},
    'OTRO_CASO': {0: random.choice(catalogs['OTRO_CASO'])},
    'TOMA_MUESTRA_LAB': {0: random.choice(catalogs['TOMA_MUESTRA_LAB'])},
    'RESULTADO_LAB': {0: random.choice(catalogs['RESULTADO_LAB'])},
    'TOMA_MUESTRA_ANTIGENO': {0: random.choice(catalogs['TOMA_MUESTRA_ANTIGENO'])},
    'RESULTADO_ANTIGENO': {0: random.choice(catalogs['RESULTADO_ANTIGENO'])},
    'CLASIFICACION_FINAL': {0: random.choice(catalogs['CLASIFICACION_FINAL'])},
    'MIGRANTE': {0: random.choice(catalogs['MIGRANTE'])},
    'PAIS_NACIONALIDAD': {0: random.choice(catalogs['PAIS_NACIONALIDAD'])},
    'PAIS_ORIGEN': {0: random.choice(catalogs['PAIS_ORIGEN'])},
    'UCI': {0: random.choice(catalogs['UCI'])},
    'AÑO_ACTUALIZACION': {0: update_date.year},
    'MES_ACTUALIZACION': {0: update_date.month},
    'DIA_ACTUALIZACION': {0: update_date.day},
    'AÑO_INGRESO': {0: ingreso_date.year},
    'MES_INGRESO': {0: ingreso_date.month},
    'DIA_INGRESO': {0: ingreso_date.day},
    'AÑO_SINTOMAS': {0: symptoms_date.year},
    'MES_SINTOMAS': {0: symptoms_date.month},
    'DIA_SINTOMAS': {0: symptoms_date.day}
}

In [16]:
# Enviar la solicitud POST al servidor Flask
response = requests.post('http://localhost:5000/predict', json=data_amigo)

# Código para imprimir predicciones del modelo o identificar errores en caso de que los haya
if response.text:
    try:
        predictions = response.json()
        print(f"Predicción del modelo: {predictions}")
    except Exception as e:
        print(f"Error al decodificar JSON: {e}")
else:
    print("Respuesta vacía del servidor")

Predicción del modelo: [2]


Por el bien de la hipocondría de mi amigo, el modelo predice que dadas sus características él no fallecería.